In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.timeanddate.com/weather/@2720383'
response = requests.get(url)
if response.status_code == 200:
    print(f"Successfully connected to {url}")

Successfully connected to https://www.timeanddate.com/weather/@2720383


In [3]:
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
# Step 1 Scarping the banner or heading
heading_tag = soup.find('h1', class_='headline-banner__title')
heading = heading_tag.get_text(strip=True) if heading_tag else "Heading not found"
print(heading)

Weather in Borlänge, Sweden


In [21]:
# Step 2: Scrape the forecast table
table = soup.find('table', id='wt-48')
    
if table:
    # Extract the day headers
    day_headers = []
    for th in table.find_all('th', colspan=True):
        day_headers.append(th.get_text(strip=True))
        
    # Extract the time period headers
        time_headers = ['Morning', 'Afternoon', 'Evening', 'Night']  # Fixed time periods

    # Combine day and time headers properly by repeating each day for 4 time periods
    headers = []
    for day in day_headers:
        for time in ['Morning', 'Afternoon', 'Evening', 'Night']:
            headers.append(f"{day} {time}")

    # Extract the table rows (weather data)
    weather_data = []
    for row in table.find_all('tr'):
        cells = []
        for td in row.find_all(['td', 'th']):
            # Check if there is an <img> tag with a title attribute for the forecast
            img_tag = td.find('img', title=True)
            if img_tag:
                # If an image tag with a title is found, use the title as the forecast
                forecast = img_tag['title']
                cells.append(forecast)
            else:
                # Otherwise, extract the text content
                text = td.get_text(strip=True).replace('\xa0', ' ')
                cells.append(text)
        if any(cells):  # Avoid empty rows
            weather_data.append(cells)

# Print headers and rows for verification
print("Headers:", headers)
for row in weather_data:
    print(row)

Headers: ['måndag Morning', 'måndag Afternoon', 'måndag Evening', 'måndag Night', 'tisdag Morning', 'tisdag Afternoon', 'tisdag Evening', 'tisdag Night']
['', 'måndag', 'tisdag', 'onsdag']
['', 'Afternoon', 'Evening', 'Night', 'Morning', 'Afternoon', 'Evening', 'Night']
['Forecast', 'Drizzle. Cloudy.', 'Sprin\xadkles. Broken clouds.', 'Clear.', 'Sunny.', 'Sunny.', 'Overcast.', 'Broken clouds.']
['Temperature', '10 °C', '5 °C', '2 °C', '1 °C', '8 °C', '3 °C', '3 °C']
['', 'Drizzle. Cloudy.', 'Sprin\xadkles. Broken clouds.', 'Clear.', 'Sunny.', 'Sunny.', 'Overcast.', 'Broken clouds.']
['Feels Like', '9 °C', '2 °C', '-1 °C', '-2 °C', '7 °C', '0 °C', '-1 °C']
['Wind Speed', '2 m/s', '3 m/s', '3 m/s', '3 m/s', '3 m/s', '3 m/s', '3 m/s']
['Wind Direction', 'NNW↑', 'NW↑', 'NW↑', 'WNW↑', 'W↑', 'SW↑', 'SW↑']
['Humidity', '79%', '83%', '82%', '80%', '52%', '80%', '91%']
['Dew Point', '6 °C', '2 °C', '-1 °C', '-2 °C', '-1 °C', '0 °C', '1 °C']
['Visibility', '5 km', '7 km', '9 km', '9 km', '12 km'

In [22]:
max_column_width = max(len(h) for h in headers)
# Prepare data for saving to a text file in a tabular format
with open("weather_data_table.txt", "w") as file:
    # Write the heading
    file.write(f"Weather Heading: {heading}\n\n")

    # Write the weather data in tabular format
    file.write("Weather Forecast:\n")
    for row in weather_data:
        row_line = "{:<{width}}".format(" ", width=max_column_width) + " | ".join([f"{r:<{max_column_width}}" for r in row]) + "\n"
        file.write(row_line)

print("Weather data successfully saved to weather_data_table.txt in tabular format")

Weather data successfully saved to weather_data_table.txt in tabular format
